In [1]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


In [2]:
# Descargar recursos adicionales de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hi4i\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hi4i\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hi4i\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Preprocesamiento de texto
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import re



In [4]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('french'))

In [2]:
import pandas as pd
import numpy as np
import re
import pandas as pd

import matcher  

In [9]:
import nltk
from nltk.tag import UnigramTagger
from nltk.stem import WordNetLemmatizer


In [10]:
# Cargar los datos del archivo CSV
df = matcher.get_df('colores.csv')

In [13]:
lemmatizer = WordNetLemmatizer()
lemmatized_colors = [lemmatizer.lemmatize(color.lower()) for color in df["colors"]]


In [14]:
train_data = [[(color, 'COLOR') for color in df["colors"]]]

In [10]:
lemmatized_colors = set(lemmatized_colors)


In [12]:
# Entrenar el etiquetador de unigramas
train_data = [[(color, 'COLOR') for color in lemmatized_colors]]
unigram_tagger = UnigramTagger(train_data)

In [14]:
def es_color(palabra):
    tagged_word = unigram_tagger.tag([palabra])
    if tagged_word[0][1] == 'COLOR':
        return True
    else:
        return False

In [16]:
es_color('rojo')

True

In [22]:
# Función para verificar si una lista de palabras contiene colores
def contiene_colores(lista_palabras):
    return [palabra for palabra in lista_palabras if es_color(palabra)]

# Ejemplo de uso
lista_palabras = ['Rojo', 'Perro', 'Azul', 'Gato']
print(contiene_colores(lista_palabras)) # Output: ['Rojo', 'Azul']

['Rojo', 'Azul']


In [ ]:
# Convertir las etiquetas de categoría a números
y = y.map({cat: i for i, cat in enumerate(categorias)})

In [3]:
df = matcher.get_df('files/origin_Madison.csv')

In [8]:
df.apply(pd.unique(df))

C:\Users\Caritoshi\AppData\Local\Temp\ipykernel_13456\1888121351.py:1: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  df.apply(pd.unique(df))


ValueError: could not broadcast input array from shape (120,20) into shape (120,)